# Intel Scene Classification : Transfer Learning + Fine Tuning

**Objective:** Use transfer learning along with fine tuning the architecture and classify the Intel Scene Images.

**Steps:**

* Load ResNet50 with out head.
* Introduce new head for the network.
* Initially, freeze the early layers of ResNet50 and warm up the new head of the network.
* Once weights settle down, unfreeze the early layers of ResNet50 and train the network.
* Predict the test images.

In [4]:
from keras.applications import ResNet50
from keras.layers import Input

In [8]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)), pooling='avg')

In [9]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

bn3a_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3a_branch2c[0][0]             
__________________________________________________________________________________________________
bn3a_branch1 (BatchNormalizatio (None, 28, 28, 512)  2048        res3a_branch1[0][0]              
__________________________________________________________________________________________________
add_20 (Add)                    (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
                                                                 bn3a_branch1[0][0]               
__________________________________________________________________________________________________
activation_62 (Activation)      (None, 28, 28, 512)  0           add_20[0][0]                     
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_62[0][0]              
__________

bn4b_branch2a (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_75 (Activation)      (None, 14, 14, 256)  0           bn4b_branch2a[0][0]              
__________________________________________________________________________________________________
res4b_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_75[0][0]              
__________________________________________________________________________________________________
bn4b_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_76 (Activation)      (None, 14, 14, 256)  0           bn4b_branch2b[0][0]              
__________________________________________________________________________________________________
res4b_bran

res5a_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_90[0][0]              
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_91 (Activation)      (None, 7, 7, 512)    0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_91[0][0]              
__________________________________________________________________________________________________
res5a_branch1 (Conv2D)          (None, 7, 7, 2048)   2099200     activation_89[0][0]              
__________________________________________________________________________________________________
bn5a_branc

In [10]:
full_model = ResNet50(weights='imagenet')

In [11]:
full_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

bn3a_branch1 (BatchNormalizatio (None, 28, 28, 512)  2048        res3a_branch1[0][0]              
__________________________________________________________________________________________________
add_36 (Add)                    (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
                                                                 bn3a_branch1[0][0]               
__________________________________________________________________________________________________
activation_111 (Activation)     (None, 28, 28, 512)  0           add_36[0][0]                     
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_111[0][0]             
__________________________________________________________________________________________________
bn3b_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2a[0][0]             
__________

bn4b_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_125 (Activation)     (None, 14, 14, 256)  0           bn4b_branch2b[0][0]              
__________________________________________________________________________________________________
res4b_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_125[0][0]             
__________________________________________________________________________________________________
bn4b_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4b_branch2c[0][0]             
__________________________________________________________________________________________________
add_41 (Add)                    (None, 14, 14, 1024) 0           bn4b_branch2c[0][0]              
                                                                 activation_123[0][0]             
__________

res5a_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_139[0][0]             
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_140 (Activation)     (None, 7, 7, 512)    0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_branch2c (Conv2D)         (None, 7, 7, 2048)   1050624     activation_140[0][0]             
__________________________________________________________________________________________________
res5a_branch1 (Conv2D)          (None, 7, 7, 2048)   2099200     activation_138[0][0]             
__________________________________________________________________________________________________
bn5a_branc

In [12]:
base_model

In [13]:
base_model.input

/input_2

In [14]:
base_model.output

mean

## Add New Head to the network

In [16]:
from keras.layers import Dense

In [15]:
head_model = base_model.output

In [17]:
NUM_CLASSES = 6

In [18]:
# add softmax layer
head_model = Dense(NUM_CLASSES, activation='softmax')(head_model)

In [19]:
head_model

Softmax.0

## Place Head Model to Base Model

In [1]:
from keras.models import Model

Using Theano backend.


In [23]:
model = Model(inputs=base_model.input, outputs=head_model)

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

bn3a_branch1 (BatchNormalizatio (None, 28, 28, 512)  2048        res3a_branch1[0][0]              
__________________________________________________________________________________________________
add_20 (Add)                    (None, 28, 28, 512)  0           bn3a_branch2c[0][0]              
                                                                 bn3a_branch1[0][0]               
__________________________________________________________________________________________________
activation_62 (Activation)      (None, 28, 28, 512)  0           add_20[0][0]                     
__________________________________________________________________________________________________
res3b_branch2a (Conv2D)         (None, 28, 28, 128)  65664       activation_62[0][0]              
__________________________________________________________________________________________________
bn3b_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3b_branch2a[0][0]             
__________

res4b_branch2b (Conv2D)         (None, 14, 14, 256)  590080      activation_75[0][0]              
__________________________________________________________________________________________________
bn4b_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_76 (Activation)      (None, 14, 14, 256)  0           bn4b_branch2b[0][0]              
__________________________________________________________________________________________________
res4b_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_76[0][0]              
__________________________________________________________________________________________________
bn4b_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4b_branch2c[0][0]             
__________________________________________________________________________________________________
add_25 (Ad

bn5a_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_90 (Activation)      (None, 7, 7, 512)    0           bn5a_branch2a[0][0]              
__________________________________________________________________________________________________
res5a_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_90[0][0]              
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_91 (Activation)      (None, 7, 7, 512)    0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_bran

## Freeze the Base Model Layers

In [25]:
len(base_model.layers)

176

In [26]:
for layer in base_model.layers:
    layer.trainable = False

In [27]:
l1 = base_model.layers[0]

In [29]:
l1.trainable

False

## Compile the Model with RMSprop

In [31]:
from keras.optimizers import RMSprop

In [32]:
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

## Load the Training Dataset

In [33]:
from my_utils.datasets import DataSetLoader
import pandas as pd

/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [34]:
IMAGE_DIR = 'dataset/train-scene classification'
TRAIN_IMAGE_LOC = IMAGE_DIR+'/train'

In [35]:
train_csv_df = pd.read_csv(IMAGE_DIR+'/train.csv')

In [42]:
train_csv_df.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [36]:
DataLoader = DataSetLoader()

In [37]:
training_files = DataLoader.get_filename_list(TRAIN_IMAGE_LOC, list(train_csv_df['image_name']))

In [38]:
training_files[0:5]

['dataset/train-scene classification/train/0.jpg',
 'dataset/train-scene classification/train/1.jpg',
 'dataset/train-scene classification/train/2.jpg',
 'dataset/train-scene classification/train/4.jpg',
 'dataset/train-scene classification/train/7.jpg']

In [39]:
import cv2

In [43]:
data = []

for file in training_files[0:120]:
    img = cv2.imread(file)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_AREA)
    
    #add to list
    data.append(img)


In [44]:
len(data)

120

In [45]:
label = train_csv_df['label'][0:120]

In [46]:
len(label)

120

## Convert the labels in binary format

In [54]:
from sklearn.preprocessing import LabelBinarizer

In [55]:
label_binary = LabelBinarizer().fit_transform(label)

In [56]:
len(label_binary)

120

In [57]:
label_binary[0:5]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [51]:
import numpy as np

In [61]:
trainX = np.array(data[0:100])
valX  = np.array(data[100:120])

trainY = np.array(label_binary[0:100])
valY   = np.array(label_binary[100:120])

In [62]:
trainY[0:3]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

## Train the Head Model

* Use 100 images to train the model

In [50]:
?model.fit

In [64]:
model.fit(trainX, trainY, batch_size=10, validation_data=(valX, valY), 
          epochs=25,  verbose=1)

#steps_per_epoch = len(trainX)// 10,

Train on 100 samples, validate on 20 samples
Epoch 1/25
100/100 [==============================] - 74s 741ms/step - loss: 1.7992 - acc: 0.3300 - val_loss: 1.3405 - val_acc: 0.4500
Epoch 2/25
100/100 [==============================] - 74s 743ms/step - loss: 0.8899 - acc: 0.6800 - val_loss: 0.8561 - val_acc: 0.7000
Epoch 3/25
100/100 [==============================] - 316s 3s/step - loss: 0.6506 - acc: 0.7900 - val_loss: 1.0697 - val_acc: 0.7000
Epoch 4/25
100/100 [==============================] - 75s 747ms/step - loss: 0.5170 - acc: 0.8400 - val_loss: 0.8549 - val_acc: 0.7000
Epoch 5/25
100/100 [==============================] - 75s 746ms/step - loss: 0.3708 - acc: 0.9200 - val_loss: 0.9386 - val_acc: 0.7000
Epoch 6/25
100/100 [==============================] - 75s 746ms/step - loss: 0.2731 - acc: 0.9500 - val_loss: 0.8102 - val_acc: 0.7000
Epoch 7/25
100/100 [==============================] - 75s 749ms/step - loss: 0.3043 - acc: 0.9300 - val_loss: 0.7452 - val_acc: 0.6500
Epoch 8/25
1

## Evaluate the Model After freezing the early layers in the Network

In [66]:
from sklearn.metrics import classification_report

In [67]:
preds = model.predict(valX)

In [69]:
preds[0:2]

array([[5.5208705e-02, 1.3848646e-04, 1.3034137e-04, 7.1708506e-05,
        1.7034564e-04, 9.4428039e-01],
       [1.6175544e-01, 1.5787469e-03, 1.1542531e-01, 3.0488473e-01,
        2.8099928e-02, 3.8825583e-01]], dtype=float32)

In [70]:
print(classification_report(valY.argmax(axis=1), preds.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         1
           3       1.00      1.00      1.00         3
           4       0.50      1.00      0.67         1
           5       0.64      1.00      0.78         9

   micro avg       0.70      0.70      0.70        20
   macro avg       0.52      0.67      0.57        20
weighted avg       0.51      0.70      0.59        20



/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Unfreeze the Early Layers

* Instead of all early layers, half of the base layers unfreeze them.

In [71]:
for layer in base_model.layers[75:]:
    layer.trainable = True

In [74]:
lay = model.layers[75]

In [75]:
lay.trainable

True

## Recompile the model

In [72]:
from keras.optimizers import SGD

In [73]:
opt = SGD(lr=0.001)

In [76]:
H = model.compile(loss="categorical_crossentropy", optimizer=opt,
                    metrics=["accuracy"])

In [77]:
model.fit(trainX, trainY, batch_size=10, validation_data=(valX, valY), 
          epochs=25,  verbose=1)

Train on 100 samples, validate on 20 samples
Epoch 1/25
100/100 [==============================] - 105s 1s/step - loss: 0.0262 - acc: 0.9900 - val_loss: 0.9079 - val_acc: 0.6500
Epoch 2/25
100/100 [==============================] - 104s 1s/step - loss: 0.0120 - acc: 1.0000 - val_loss: 0.8684 - val_acc: 0.7000
Epoch 3/25
100/100 [==============================] - 105s 1s/step - loss: 0.0100 - acc: 1.0000 - val_loss: 0.8330 - val_acc: 0.7000
Epoch 4/25
100/100 [==============================] - 104s 1s/step - loss: 0.0112 - acc: 1.0000 - val_loss: 0.7980 - val_acc: 0.7000
Epoch 5/25
100/100 [==============================] - 105s 1s/step - loss: 0.0138 - acc: 1.0000 - val_loss: 0.7735 - val_acc: 0.7000
Epoch 6/25
100/100 [==============================] - 406s 4s/step - loss: 0.0115 - acc: 1.0000 - val_loss: 0.7465 - val_acc: 0.7000
Epoch 7/25
100/100 [==============================] - 108s 1s/step - loss: 0.0120 - acc: 1.0000 - val_loss: 0.7274 - val_acc: 0.7000
Epoch 8/25
100/100 [====

In [78]:
preds = model.predict(valX)

In [79]:
print(classification_report(valY.argmax(axis=1), preds.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         1
           3       0.67      0.67      0.67         3
           4       0.50      1.00      0.67         1
           5       0.82      1.00      0.90         9

   micro avg       0.75      0.75      0.75        20
   macro avg       0.66      0.68      0.63        20
weighted avg       0.79      0.75      0.73        20



In [80]:
model.save('./model/small_TL_Fine_Tune.hdf5')